In [11]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from datetime import datetime

In [46]:
# load all files to single df
load_dotenv()
zip_file_name = os.getenv('OS_FILENAME')
zip_dir = zip_file_name.replace('.zip', '')
csv_file_names = sorted(os.listdir(zip_dir))

master_df = pd.DataFrame()
for file_name in csv_file_names[:50]:
    print(f'file_name: [{file_name}], time: [{datetime.now().strftime("%H:%M:%S")}]')
    df = pd.read_csv(f'{zip_dir}/{file_name}', header=None, names=range(30), dtype=object)
    master_df = pd.concat([master_df, df])

print('done')

file_name: [AddressBasePremium_FULL_2024-05-29_001.csv], time: [14:59:14]
file_name: [AddressBasePremium_FULL_2024-05-29_002.csv], time: [14:59:17]
file_name: [AddressBasePremium_FULL_2024-05-29_003.csv], time: [14:59:19]
file_name: [AddressBasePremium_FULL_2024-05-29_004.csv], time: [14:59:21]
file_name: [AddressBasePremium_FULL_2024-05-29_005.csv], time: [14:59:22]
file_name: [AddressBasePremium_FULL_2024-05-29_006.csv], time: [14:59:24]
file_name: [AddressBasePremium_FULL_2024-05-29_007.csv], time: [14:59:27]
file_name: [AddressBasePremium_FULL_2024-05-29_008.csv], time: [14:59:30]
file_name: [AddressBasePremium_FULL_2024-05-29_009.csv], time: [14:59:33]
file_name: [AddressBasePremium_FULL_2024-05-29_010.csv], time: [14:59:36]
file_name: [AddressBasePremium_FULL_2024-05-29_011.csv], time: [14:59:39]
file_name: [AddressBasePremium_FULL_2024-05-29_012.csv], time: [14:59:42]
file_name: [AddressBasePremium_FULL_2024-05-29_013.csv], time: [14:59:46]
file_name: [AddressBasePremium_FULL_20

In [47]:
# build individual dfs
rd = {}
record_types = [11, 15, 21, 23, 24, 28, 30, 31, 32]

for record_type in record_types:
    print(f'record_type: [{record_type}], time: [{datetime.now().strftime("%H:%M:%S")}], count: {len(master_df[master_df[0].astype(int) == record_type])}')
    rd[record_type] = master_df[master_df[0].astype(int) == record_type]

record_type: [11], time: [15:09:08], count: 1507301
record_type: [15], time: [15:09:11], count: 1630431
record_type: [21], time: [15:09:15], count: 40825714
record_type: [23], time: [15:09:55], count: 5174286
record_type: [24], time: [15:10:01], count: 0
record_type: [28], time: [15:10:03], count: 0
record_type: [30], time: [15:10:06], count: 0
record_type: [31], time: [15:10:09], count: 0
record_type: [32], time: [15:10:11], count: 0


In [23]:
# add headings
type_headings = {
    11:['record_identifier', 'change_type', 'pro_order', 'usrn', 'record_type', 'swa_org_ref_naming', 'state', 'state_date', 'street_surface', 'street_classification', 'version', 'street_start_date', 'street_end_date', 'last_update_date', 'record_entry_date', 'street_start_x', 'street_start_y', 'street_start_lat', 'street_start_long', 'street_end_x', 'street_end_y', 'street_end_lat', 'street_end_long', 'street_tolerance'],
    15:['record_identifier', 'change_type', 'pro_order', 'usrn', 'street_description', 'locality', 'town_name', 'administrative_area', 'language', 'start_date', 'end_date', 'last_update_date', 'entry_date'],
    21:['record_identifier', 'change_type', 'pro_order', 'uprn', 'logical_status', 'blpu_state', 'blpu_state_date', 'parent_uprn', 'x_coordinate', 'y_coordinate', 'latitude', 'longitude', 'rpc', 'local_custodian_code', 'country', 'start_date', 'end_date', 'last_update_date', 'entry_date', 'addressbase_postal', 'postcode_locator', 'multi_occ_count'],
    23:['record_identifier', 'change_type', 'pro_order', 'uprn', 'xref_key', 'cross_reference', 'version', 'source', 'start_date', 'end_date', 'last_update_date', 'entry_date'],
    24:['record_identifier', 'change_type', 'pro_order', 'uprn', 'lpi_key', 'language', 'logical_status', 'start_date', 'end_date', 'last_update_date', 'entry_date', 'sao_start_number', 'sao_start_suffix', 'sao_end_number', 'sao_end_suffix', 'sao_text', 'pao_start_number', 'pao_start_suffix', 'pao_end_number', 'pao_end_suffix', 'pao_text', 'usrn', 'usrn_match_indicator', 'area_name', 'level', 'official_flag'],
    28:['record_identifier', 'change_type', 'pro_order', 'uprn', 'udprn', 'organisation_name', 'department_name', 'sub_building_name', 'building_name', 'building_number', 'dependant_thoroughfare', 'thoroughfare', 'double_dependant_locality', 'dependant_locality', 'post_town', 'postcode', 'postcode_type', 'delivery_point_suffix', 'welsh_dependant_thoroughfare', 'welsh_thoroughfare', 'welsh_double_dependant_locality', 'welsh_dependant_locality', 'welsh_post_town', 'po_box_number', 'process_date', 'start_date', 'end_date', 'last_update_date', 'entry_date'],
    30:['record_identifier', 'change_type', 'pro_order', 'uprn', 'succ_key', 'start_date', 'end_date', 'last_update_date', 'entry_date', 'successor'],
    31:['record_identifier', 'change_type', 'pro_order', 'uprn', 'org_key', 'organisation', 'legal_name', 'start_date', 'end_date', 'last_update_date', 'entry_date'],
    32:['record_identifier', 'change_type', 'pro_order', 'uprn', 'class_key', 'classification_code', 'class_scheme', 'scheme_version', 'start_date', 'end_date', 'last_update_date', 'entry_date']
}

for record_type in rd.keys():
    print(f'type: [{record_type}], time: [{datetime.now().strftime("%H:%M:%S")}]')
    rd[record_type].columns = type_headings[record_type]

type: [11], time: [14:26:35]


ValueError: Length mismatch: Expected axis has 30 elements, new values have 24 elements

In [1]:
# keys
type_keys = {
    11:['usrn'],
    15:['usrn'],
    21:['uprn'],
    23:['xref_key'],
    24:['lpi_key'],
    28:['udprn'],
    30:['succ_key'],
    31:['org_key'],
    32:['class_key']
}

for record_type in rd.keys():
    print(f'type: [{record_type}], time: [{datetime.now().strftime("%H:%M:%S")}]')
    rd[record_type].set_index(type_keys[record_type])

NameError: name 'rd' is not defined

In [ ]:
# joins
refs_with_uarn = rd[23][rd[23]['source']=='7666VN']
blpu_with_uarn = rd[21].join(refs_with_uarn, on='uprn', how='outer')

In [24]:
rd[21][1000:1100]

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29


In [48]:
for record_type in rd.keys():
    print(f'type: [{record_type}], count: [{len(rd[record_type])}]')

type: [11], count: [1507301]
type: [15], count: [1630431]
type: [21], count: [40825714]
type: [23], count: [5174286]
type: [24], count: [0]
type: [28], count: [0]
type: [30], count: [0]
type: [31], count: [0]
type: [32], count: [0]
